<p style="font-weight:bold;"> <span style="font-size: 36px"> Scenario Transactional Data Import Test </span> </p>

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [0]:
DataSource.Reset(x => x.ResetType<RawVariable>().ResetType<IfrsVariable>());

In [0]:
static ImportStorage mystorage;
static ImportIdentity[] myidentities; 
static IModel mymodel;
static List<string> my = new List<string>();

Import.DefineFormat(ImportFormats.Cashflow, async (options, dataSet) => {
    Activity.Start();
    my.Add("0");
    var mainArgs = (await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(dataSet)) with {ImportFormat = ImportFormats.Cashflow};
    if(Activity.HasErrors()) return Activity.Finish();
    my.Add("1");
    await DataNodeFactoryAsync(dataSet, ImportFormats.Cashflow, mainArgs);
    my.Add("2");
    if(Activity.HasErrors()) return Activity.Finish();
    
    var workspace = Workspace.CreateNew();
    var log = await ParseCashflowsToWorkspaceAsync(dataSet, mainArgs, workspace);
    if(log.Errors.Any()) return Activity.Finish().Merge(log);

    var targetArgs = mainArgs.Scenario == null 
                     ? mainArgs.RepeatOnce().Concat(await DataSource.Query<PartitionByReportingNodeAndPeriod>() 
                               .Where(x => x.ReportingNode == mainArgs.ReportingNode && x.Year == mainArgs.Year && x.Month == mainArgs.Month && x.Scenario != null)
                               .Select(x => new ImportArgs(x.ReportingNode, x.Year, x.Month, default(Periodicity), x.Scenario, ImportFormats.Cashflow)).ToArrayAsync())
                     : mainArgs.RepeatOnce();
    
    foreach (var args in targetArgs) {
        log = log.Merge(await CalculateAndUploadIfrsVariablesAsync(args, workspace));
        if(log.Errors.Any()) return Activity.Finish().Merge(log);

        var storage = new ImportStorage(args, DataSource, workspace);
        await storage.InitializeAsync();
        var universe = Scopes.ForStorage(storage).ToScope<IModel>();
        var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
        var ivs = universe.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.CalculatedIfrsVariables);
        mystorage = storage; 
        myidentities = identities.ToArray(); 
        mymodel = universe;

        if(args.Scenario == mainArgs.Scenario) await CommitToDatabase<RawVariable, PartitionByReportingNodeAndPeriod>(workspace, (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(args)));
    }

    return Activity.Finish().Merge(log); 
});

In [0]:
var actualEstimateTypes = new[] {"A","AA","OA","DA"};

In [0]:
var argsBestEstimate = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsScenario = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "Test", ImportFormats.Cashflow);

# Initialization

In [0]:
await Import.FromFile("../Files/TransactionalData/NominalCashflows_CH_2020_12.csv").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromFile("../Files/TransactionalData/Actuals_CH_2020_12.csv").WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
var ivsBestEstimateInit = await DataSource.Query<IfrsVariable>().ToArrayAsync();

# Scenario Cash flows

In [0]:
var cashflowsScenarioTest = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,CL,C,,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,CU,CL,C,,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-3.3
DT1.1,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";

In [0]:
await Import.FromString(cashflowsScenarioTest).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioInitCashFlows = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioInitCashFlows.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());

In [0]:
diffs.Where(x => actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().Be(0);
diffs.Where(x => !actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);
ivsScenarioInitCashFlows.Where(x => x.EstimateType == "DA").ToArray().Length.Should().NotBe(0);

# Scenario Actuals

In [0]:
var actualsScenarioTest = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,-308";

In [0]:
await Import.FromString(actualsScenarioTest).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioInitActuals = DataSource.Query<IfrsVariable>().ToArray();
var diffs = ivsScenarioInitActuals.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());

In [0]:
diffs.Where(x => actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);
diffs.Where(x => x.EstimateType == "CU").ToArray().Length.Should().Be(0);
ivsScenarioInitActuals.Where(x => x.EstimateType == "DA").ToArray().Length.Should().Be(0);

# Robustness: back to Best Estimate

In [0]:
var cashflowsScenarioTestEqualToBestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0
DT1.1,NIC,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,CU,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5";

In [0]:
var actualsScenarioTestEqualToBestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,-280";

In [0]:
await Import.FromString(actualsScenarioTestEqualToBestEstimate).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromString(cashflowsScenarioTestEqualToBestEstimate).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
var ivsScenarioReimportEqualToBestEstimate = await DataSource.Query<IfrsVariable>().ToArrayAsync(); 
ivsScenarioReimportEqualToBestEstimate.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance()).Count().Should().Be(0);

In [0]:
// TODO: why here I have DA if I imported cashflow at the end
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
DataSource.Query<IfrsVariable>().Where(x => x.EstimateType == "DA" && x.DataNode == "DT1.1")

# Reimport of Best Estimate Cashflow

In [0]:
var newCashflowsBestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,210,0,0,210,0,0,210,0,0,210,0,0,0,210,0,0,210,0,0,210,0,0,210,0
DT1.1,NIC,CL,C,,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37
DT1.1,CU,CL,C,,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-3.3
DT1.1,RA,CL,C,,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42";

In [0]:
await Import.FromString(newCashflowsBestEstimate).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateReimportCashFlows = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsBestEstimateReimportCashFlows.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());
var items = diffs.Where(x => actualEstimateTypes.Contains(x.EstimateType)).Select(x => x.EstimateType).Distinct().ToArray();

In [0]:
items.Length.Should().Be(1);
(items.First() == "DA").Should().Be(true);
diffs.Where(x => !actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportCashFlows = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioReimportCashFlows.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());
var items = diffs.Where(x => actualEstimateTypes.Contains(x.EstimateType)).Select(x => x.EstimateType).Distinct().ToArray();

In [0]:
diffs.Where(x => !actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);
items.Length.Should().Be(0);
//(items.First() == "DA").Should().Be(true);   // TODO: this shouldn't exist

# Reimport of Best Estimate Actuals

In [0]:
var newActualsBestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,-2000";

In [0]:
await Import.FromString(newActualsBestEstimate).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
DataSource.Query<IfrsVariable>().Where(x => x.EstimateType == "DA" && x.DataNode == "DT1.1")

In [0]:
// TODO: why this is not empty?
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
DataSource.Query<IfrsVariable>().Where(x => x.EstimateType == "DA" && x.DataNode == "DT1.1")

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateReimportActuals = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsBestEstimateReimportActuals.Except(ivsBestEstimateReimportCashFlows, IfrsVariableComparer.Instance());

In [0]:
diffs.Where(x => actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);
diffs.Where(x => x.EstimateType == "CU").ToArray().Length.Should().Be(0);

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportActuals = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioReimportActuals.Except(ivsBestEstimateReimportCashFlows, IfrsVariableComparer.Instance());

In [0]:
diffs.Where(x => actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);
diffs.Where(x => x.EstimateType == "CU").ToArray().Length.Should().Be(0);

# Update of Scenario Deferrable Actuals via Best Estimate Coverage Units

In [0]:
var cashflowsScenarioNoCU = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,CL,C,,-274.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";

In [0]:
var newCashflowsBestEstimateBis = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,200,0,0,200,0,0,200,0,0,200,0,0,0,200,0,0,200,0,0,200,0,0,200,0
DT1.1,NIC,CL,C,,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33
DT1.1,CU,CL,C,,-434,-23,-23,-23,-23,-23,-23,-23,-23,-23,-23,-23,-23,-19,-19,-19,-19,-19,-19,-19,-19,-19,-19,-3.5
DT1.1,RA,CL,C,,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45";

In [0]:
await Import.FromString(cashflowsScenarioNoCU).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportCashflowNoCU = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
ivsScenarioReimportCashflowNoCU.Where(x => x.AocType == "AM")

In [0]:
await Import.FromString(newCashflowsBestEstimateBis).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportCashFlowsWithCU = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioReimportCashFlowsWithCU.Except(ivsScenarioReimportCashflowNoCU, IfrsVariableComparer.Instance());
var items = diffs.Where(x => actualEstimateTypes.Contains(x.EstimateType)).Select(x => x.EstimateType).Distinct().ToArray();

In [0]:
ivsScenarioReimportCashFlowsWithCU.Where(x => x.AocType == "AM")

In [0]:
diffs.Where(x => !actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);
items.Length.Should().Be(1);
(items.First() == "DA").Should().Be(true);

In [0]:
mymodel.GetScopes<ComputeAllScopes>(myidentities).SelectMany(x => x.DeferrableActual)//.Select(x => (x.Value, x.Identity))

In [0]:
mymodel.GetScopes<DeferrableActual>(myidentities).Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                   DataNode = x.Identity.DataNode,
                                   AocType = x.Identity.AocType,
                                   Novelty = x.Identity.Novelty,
                                   AccidentYear = null,
                                   Value = x.Value,
                                   Partition = mystorage.TargetPartition
                                })

In [0]:
myidentities.Where(x => x.AocType == "AM")

In [0]:
//    private double AmortizationFactor => GetScope<CurrentPeriodAmortizationFactor>(Identity, o => o.WithContext(EconomicBasis)).Value;
//    private double AggregatedDeferrable => GetScope<PreviousAocSteps>((Identity, InputSource.Actual)).Values
//                                            .Sum(aocStep => GetScope<DeferrableActual>(Identity with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}).Value);
//    double DeferrableActual.Value => -1d * AggregatedDeferrable * AmortizationFactor;

var amids = myidentities.Where(x => x.AocType == "AM").ToArray();
mymodel.GetScopes<CurrentPeriodAmortizationFactor>(amids, o => o.WithContext("L")).Select(x => (x.EstimateType, x.Value, x.Identity))

In [0]:
mymodel.GetScopes<PreviousAocSteps>(amids.Select(x => (x, InputSource.Actual))).Select(x => (x.Values, x.Identity))
    //.Select(x => x.Values.Sum( aocStep => mymodel.GetScope<DeferrableActual>(Identity with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}).Value )
    //.Select(x => (x.EstimateType, x.Value, x.Identity))

In [0]:
mymodel.GetScopes<PreviousAocSteps>(amids.Select(x => (x, InputSource.Actual))).Select(x => (x.Values, x.Identity))
    .Select((x,i) => x.Values.Sum( aocStep => mymodel.GetScope<DeferrableActual>(amids[i] with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}).Value ) )